#  주택청약 FAQ 시스템 챗봇 구현 
- 문서 전처리 + RAG + Gradio ChatInterface

### **학습 목표:**  RAG 기반의 주택청약 FAQ 시스템을 Gradio로 구현한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.9, 
)

/Users/kaydash/miniconda3/envs/modu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **문서 전처리 파이프라인**

* 문서 전처리의 첫 단계는 데이터 정제로, 원본 문서에서 불필요한 요소(HTML 태그, 특수문자, 중복 내용 등)를 제거하고 텍스트를 표준화하는 과정입니다. 이는 검색 품질과 직결되는 중요한 단계입니다.

* 문서 청킹(Chunking)은 긴 문서를 의미 있는 작은 단위로 분할하는 과정으로, 문장 단위나 단락 단위로 나누되 문맥의 연속성을 유지하는 것이 핵심입니다. 이는 검색 정확도와 답변 생성의 품질에 직접적인 영향을 미칩니다.

* 임베딩(Embedding) 생성은 텍스트를 고차원의 벡터로 변환하는 과정으로, 문서의 의미적 특성을 수치화하여 효율적인 검색을 가능하게 합니다. 이때 사용되는 임베딩 모델의 선택이 검색 성능을 좌우하는 중요한 요소가 됩니다.

* 마지막으로 벡터 데이터베이스 색인화 단계에서는 생성된 임베딩을 효율적으로 저장하고 검색할 수 있는 구조로 변환합니다. 이는 대규모 문서 집합에서도 빠른 검색을 가능하게 하는 핵심 요소입니다.


### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공
- Q1 ~ Q50까지 모두 50개의 문답이 포함된 텍스트 파일

In [4]:
# 파일 경로 설정
faq_text_file = "data/housing_faq.txt"

# 파일 읽기 - 파이썬 내장 함수 사용
with open(faq_text_file, 'r') as f:
    faq_text = f.read()

# 파일 내용 확인
print(faq_text[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


##### ***[실습] TextLoader를 사용하여, 텍스트 문서를 로드합니다.*** 

In [5]:
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 문서 로드
loader = TextLoader(faq_text_file)
docs = loader.load()

# 문서 개수 확인
print(f"로드된 문서 개수: {len(docs)}")

로드된 문서 개수: 1


In [6]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [7]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': 'data/housing_faq.txt'}

### 2) 문서 전처리

`(1) 각 질문과 답변을 쌍으로 추출하여 정리 (정규표현식 활용)`

In [8]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [9]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


`(2) LLM으로 추가 정보를 추출`
- 텍스트에서 키워드와 핵심 개념을 추출하는 체인
- 메타데이터 or 본문(page_content)에 추가하여 검색에 활용    

##### ***[실습] 다음 모델 스키마를 사용하여, 키워드와 요약을 추출하는 체인을 완성합니다.*** 

In [10]:
# QA 쌍 확인 (첫번째)
print((qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer']))

경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?

해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약")

# 프롬프트 템플릿 정의
template = """다음 텍스트에서 가장 중요한 키워드와 간단한 요약을 추출해주세요:

텍스트:
{text}

키워드는 법률용어, 주제 등 검색에 유용한 단어나 구절이어야 합니다.
요약은 텍스트의 핵심 내용을 간결하게 나타내야 합니다.
"""

# LCEL 체인 구성 (Sturctured Output 사용)
prompt = ChatPromptTemplate.from_template(template)
llm_with_structure = llm.with_structured_output(KeywordOutput)
keyowrd_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke({"text": qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer']})
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 해당 주택건설지역, 경기도 과천시, 행정구역, 특별시, 광역시, 특별자치시, 특별자치도, 시, 군
요약: 해당 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 지역에 해당한다.


`(3) QA 쌍을 문자열 포맷팅하고 문서 객체로 변환`

In [12]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # QA 쌍을 포맷팅
        formatted_output = (
            f"[{pair['number']}]\n"
            f"질문: {pair['question']}\n"
            f"답변: {pair['answer']}\n"
        )

        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=formatted_output,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역, 경기도 과천시, 행정구역, 특별시, 광역시, 특별자치시, 특별자치도, 시, 군',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1,
 'summary': '해당 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 '
            '공급되는 주택의 경우 과천시가 해당 지역에 

##### ***[실습] 문서 객체를 포맷팅하여 구성합니다.*** 

- 요약문을 시맨틱 검색에 활용합니다. 다음 구조로 문서 객체를 생성합니다. 
    - page_content: 요약
    - metadata: 기타 정보

In [13]:
def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    - page_content: 요약
    - metadata: 기타 정보
    """
    processed_docs = []
    for pair in qa_pairs:
        # QA 쌍을 포맷팅
        qa_text = f"{pair['question']}\n\n{pair['answer']}"
        
        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke({"text": qa_text})
        
        # 문서 객체 생성 - page_content에 요약 정보를 저장
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'full_content': f"[{pair['number']}]\n질문: {pair['question']}\n답변: {pair['answer']}"
            }
        )
        processed_docs.append(doc)

    return processed_docs

In [14]:
# QA 쌍 포맷팅
summary_formatted_docs = format_qa_pairs_with_summary(qa_pairs) 
print(f"포맷팅된 문서 개수: {len(summary_formatted_docs)}")

# 문서 확인
print(summary_formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
해당 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 지역에 해당한다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'full_content': '[1]\n'
                 '질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?\n'
                 '답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 '
                 '특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 '
                 '과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, '
                 '인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역, 경기도 과천시, 특별시, 광역시

# 벡터 저장 

In [15]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장
vector_store = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_db",
    persist_directory="./chroma_db",
)

##### ***[실습] 요약 문서(summary_formatted_docs)를 벡터 스토어에 저장합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [16]:
# 요약 문서 벡터 저장
vector_store_summary = Chroma.from_documents(
    documents=summary_formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_summary_db",
    persist_directory="./chroma_summary_db",
)

# 문서 검색

In [17]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

##### ***[실습] 앞에서 저장한 벡터 스토어를 로드합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [18]:
# 요약 벡터 저장소 로드
vector_store_with_summary = Chroma(
    collection_name="housing_faq_summary_db",
    persist_directory="./chroma_summary_db", 
    embedding_function=embeddings,
)

In [19]:
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급 지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다

##### ***[실습] MMR 검색기를 정의합니다.*** 

- 요약 문서 벡터 스토어를 사용
- 10개의 문서를 가져와서, 다양성 기반으로 3개를 선택 (다양성은 중간 수준 적용)

In [20]:
# MMR 검색기 정의 - 다양성 기반 검색
mmr_retriever = vector_store_with_summary.as_retriever(
    search_type="mmr",  # 다양성 기반 검색 
    search_kwargs={
        "k": 3,         # 최종적으로 반환할 문서 수
        "fetch_k": 10,  # 초기에 가져올 문서 수
        "lambda_mult": 0.5,  # 다양성 가중치 (0: 다양성 최대, 1: 관련성 최대)
    }
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

해당 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 지역에 해당한다.
--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 시, 군, 행정구역
1
경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.
무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하지 않은 세대의 구성원을 의미한다.
--------------------------------------------------
무주택세대구성원, 청약신청자, 세대주, 주택 소유 여부
47
무주택세대구성원이란?
무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.
인천시에 거주하는 청약신청자가 서울에서 전용면적 85㎡ 이하 민영주택을 청약할 경우 예치기준금액은 250만원이며, 이는 주택공급에 관한 규칙 별표2에 따라 주민등록 거주지와 주택 평형에 따라 결정된다.
--------------------------------------------------
민영주택 청약 예치기준금액, 주민등록 거주지, 전용면적 85㎡ 이하, 인천시, 주택공급에 관한 규칙 별표2
38
주민등록 거주지가 인천시인 청약신청자가 서울에서 분양하는 전용면적 85㎡이하의 민영주택을 청약코자 할 때 예치기준금액은?
주택공급에 관한 규칙 [별표2]에서 민영주택 청약 예치기준금액을 규정하고

### **[심화] 메타데이터 기반 필터링**

- Chroma 문서: https://docs.trychroma.com/docs/querying-collections/metadata-filtering

In [21]:
# 단일 필드 정확히 일치
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": "주택건설지역"}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [22]:
# $eq 연산자 사용 - 정확히 일치

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$eq": "주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [23]:
# $ne (Not Equal) 연산자 사용 - 정확히 일치하지 않는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$ne": "주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급 지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다

In [24]:
# $in 연산자로 여러 값 중 일치하는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$in": ["주택건설지역", "청약예금"]}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [25]:
# 숫자 범위 검색 ($gt, $gte, $lt, $lte) - question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"question_id": {"$gte": 10}}},
)

query = "무주택자 기준은 무엇인가요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원, 세대분리, 주민등록표, 무주택자, 유주택자, 직계비속 배우자, 세대원 범위
50
[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원, 세대분리, 주민등록표, 직계비속 배우자, 주택소유
50
[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원, 세대분리, 주민등록표, 무주택자, 유주택자, 직계비속 배우자, 세대원 범위
50
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------

In [26]:
# $and로 여러 조건 조합 - keyword가 "주택건설지역"이고 question_id가 10 미만인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$and": [
        {"keyword": "주택건설지역"}, 
        {"question_id": {"$lt": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [27]:
# $or로 여러 조건 중 하나 일치하는 문서 검색 - keyword가 "주택건설지역"이거나 question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$or": [
        {"keyword": "주택건설지역"}, 
        {"question_id": {"$gte": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원, 청약신청자, 세대주, 주택 소유 여부
47
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원, 청약신청자, 세대원, 주택 소유 여부, 세대주
47
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원, 청약, 주택 소유 여부, 세대주
47
[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원, 세대분리, 주민등록표, 직계비속 배우자, 주택소유
50


In [28]:
# 정규식 패턴 매칭 - page_content 본문에 "주택건설지역"이 포함된 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={'where_document': {'$contains': '주택건설지역'}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급 지역
1
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다

##### ***[실습] 메타데이터 필터링 조건을 적용하는 실습을 수행합니다..*** 

- 요약 문서 벡터 스토어 기반 MMR 검색기에 적용 

In [29]:
# 메타데이터 필터링 조건을 적용한 MMR 검색기
filtered_mmr_retriever = vector_store_with_summary.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "fetch_k": 10,
        "lambda_mult": 0.5,
        "filter": {"keyword": {"$eq": "주택건설지역"}}  # 키워드 필터링 예시
    }
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = filtered_mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional

class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    question_id: Optional[int] = Field(description="질문 ID의 최소값")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
다음 예시를 참조한다:

1. 키워드 검색 예시:
- 입력: "주택건설 관련 문서 찾아줘"
출력:
keyword: "주택건설"
keyword_expression: "$eq"

2. 질문 ID 검색 예시:
- 입력: "질문 ID 10번 이상인 문서"
출력:
question_id: 10
question_id_expression: "$gte"

3. 복합 검색 예시:
- 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
출력:
keyword: "주택건설"
keyword_expression: "$eq"
question_id: 5 
question_id_expression: "$gte"

검색 표현식은 다음과 같이 사용한다:
- 동등 비교: $eq
- 크거나 같음: $gte
- 작거나 같음: $lte
- 범위 검색: $gt, $lt

요청에 해당 정보가 없는 경우 해당 필드는 null로 반환한다.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{query}")
])

# 구조화된 출력을 위한 체인 생성
model_with_structure = llm.with_structured_output(MetadataFilter)
metadata_chain = prompt | model_with_structure

# 사용 예시
query = "주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"
filter_params = metadata_chain.invoke({"query": query})

print(f"키워드: {filter_params.keyword}")
print(f"키워드 표현식: {filter_params.keyword_expression}")
print(f"질문 ID: {filter_params.question_id}")
print(f"질문 ID 표현식: {filter_params.question_id_expression}")

키워드: 주택건설지역
키워드 표현식: $eq
질문 ID: 10
질문 ID 표현식: $lte


In [31]:
# 메타데이터 필터 생성
filter_dict = {}

if filter_params.keyword and filter_params.question_id:
    # 두 조건 모두 있는 경우 AND 연산자 사용
    filter_dict = {
        "$and": [
            {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
            {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
        ]
    }
elif filter_params.keyword:
    # 키워드 조건만 있는 경우
    filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
elif filter_params.question_id:
    # 질문 ID 조건만 있는 경우
    filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

# retriever에 필터 적용 
retriever = vector_store.as_retriever(
    search_kwargs={"filter": filter_dict} if filter_dict else {}
)

# 검색 실행
query = "주택건설지역 관련 문서를 질문 ID 10번 이하인 문서중에서 검색해주세요"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from pydantic import BaseModel, Field
from typing import Optional, Dict


class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    question_id: Optional[int] = Field(description="질문 ID의 최소값")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


def create_metadata_filter(query: str, llm) -> Dict:
    """
    사용자 쿼리를 분석하여 Chroma DB 검색을 위한 메타데이터 필터를 생성합니다.

    Args:
        query: 사용자 검색 쿼리.
        llm: 사용할 언어 모델.

    Returns:
        Chroma DB 검색에 사용할 수 있는 필터 딕셔너리.
    """
    system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
    다음 예시를 참조한다:

    1. 키워드 검색 예시:
    - 입력: "주택건설 관련 문서 찾아줘"
    출력:
    keyword: "주택건설"
    keyword_expression: "$eq"

    2. 질문 ID 검색 예시:
    - 입력: "질문 ID 10번 이상인 문서"
    출력:
    question_id: 10
    question_id_expression: "$gte"

    3. 복합 검색 예시:
    - 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
    출력:
    keyword: "주택건설"
    keyword_expression: "$eq"
    question_id: 5
    question_id_expression: "$gte"

    검색 표현식은 다음과 같이 사용한다:
    - 동등 비교: $eq
    - 크거나 같음: $gte
    - 작거나 같음: $lte
    - 범위 검색: $gt, $lt

    요청에 해당 정보가 없는 경우 해당 필드는 null로 반환한다.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])

    model_with_structure = llm.with_structured_output(MetadataFilter)
    metadata_chain = prompt | model_with_structure

    filter_params = metadata_chain.invoke({"query": query})

    filter_dict = {}

    if filter_params.keyword and filter_params.question_id:
        filter_dict = {
            "$and": [
                {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
                {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
            ]
        }
    elif filter_params.keyword:
        filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
    elif filter_params.question_id:
        filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

    return filter_dict



# 사용 예시 
query = "주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"

@chain
def metadata_filter_query(query: str):

    llm = ChatOpenAI(
        model='gpt-4.1-mini',
        temperature=0.1,
        top_p=0.9,
    )

    filter_dict = create_metadata_filter(query, llm)
    print(filter_dict)

    retriever = vector_store.as_retriever(
            search_kwargs={"filter": filter_dict} if filter_dict else {}
    )
    results = retriever.invoke(query)

    return results

results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

{'$and': [{'keyword': {'$eq': '주택건설지역'}}, {'question_id': {'$lte': 10}}]}


In [33]:
# 다른 쿼리
query = "청약통장에 대한 정보를 찾아주세요"
results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

{'keyword': {'$eq': '청약통장'}}


##### ***[실습] 메타데이터 필터링 체인의 성능을 개선합니다.*** 

- (예시)
    - 추가 예시 제공을 통해 필터링 적용 법위 확대
    - 복합조건식 추출 기능을 추가 등

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional, List

# 개선된 메타데이터 필터 스키마 - 범위 검색 및 복합 쿼리 지원
class ImprovedMetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식 ($eq, $ne, $in)")
    keyword_list: Optional[List[str]] = Field(description="여러 키워드 검색을 위한 목록 ($in 연산자와 함께 사용)")
    question_id_min: Optional[int] = Field(description="질문 ID 최소값")
    question_id_max: Optional[int] = Field(description="질문 ID 최대값")
    question_contains: Optional[str] = Field(description="질문에 포함된 문자열")
    answer_contains: Optional[str] = Field(description="답변에 포함된 문자열")
    operation: Optional[str] = Field(description="조건간 연산 ($and, $or)")

# 개선된 프롬프트
improved_system_prompt = """사용자 쿼리에서 검색 조건을 추출하여 Chroma DB 메타데이터 필터를 생성합니다.
조건을 최대한 세밀하게 추출하고, 다음 출력 형식으로 반환하세요:

1. 키워드 검색:
   - keyword: 검색할 키워드
   - keyword_expression: 검색 연산자 ($eq, $ne, $in)
   - keyword_list: 여러 키워드를 검색할 때 사용하는 목록 ($in 연산자와 함께 사용)

2. 질문 ID 범위 검색:
   - question_id_min: 최소 질문 ID
   - question_id_max: 최대 질문 ID

3. 텍스트 내용 검색:
   - question_contains: 질문에 포함된 문자열
   - answer_contains: 답변에 포함된 문자열

4. 조건 조합:
   - operation: 여러 조건을 조합하는 연산자 ($and, $or)

예시:
- "청약통장과 주택건설지역 관련 문서를 찾아줘"
  → keyword_list: ["청약통장", "주택건설지역"], keyword_expression: "$in"

- "질문 ID 5번부터 15번 사이의 청약통장 관련 문서"
  → keyword: "청약통장", keyword_expression: "$eq", question_id_min: 5, question_id_max: 15, operation: "$and"

- "무주택 관련 문서 중에 답변에 '세대' 단어가 포함된 것"
  → keyword: "무주택", keyword_expression: "$eq", answer_contains: "세대", operation: "$and"

요청에 해당 정보가 없는 경우 해당 필드는 null로 반환합니다.
"""

# LCEL 체인 구성
improved_prompt = ChatPromptTemplate.from_messages([
    ("system", improved_system_prompt),
    ("human", "{query}")
])

improved_model_with_structure = llm.with_structured_output(ImprovedMetadataFilter)
improved_metadata_chain = improved_prompt | improved_model_with_structure

@chain
def improved_metadata_filter_query(query: str):
    # 필터 파라미터 추출
    filter_params = improved_metadata_chain.invoke({"query": query})
    print(f"추출된 필터 파라미터: {filter_params}")
    
    # 필터 조건 생성
    filter_dict = {}
    conditions = []
    
    # 키워드 조건 처리
    if filter_params.keyword:
        if filter_params.keyword_expression == "$eq" or not filter_params.keyword_expression:
            conditions.append({"keyword": filter_params.keyword})
        elif filter_params.keyword_expression == "$ne":
            conditions.append({"keyword": {"$ne": filter_params.keyword}})
    
    # 키워드 목록 처리
    if filter_params.keyword_list and len(filter_params.keyword_list) > 0:
        conditions.append({"keyword": {"$in": filter_params.keyword_list}})
    
    # 질문 ID 범위 처리
    id_conditions = []
    if filter_params.question_id_min is not None:
        id_conditions.append({"question_id": {"$gte": filter_params.question_id_min}})
    if filter_params.question_id_max is not None:
        id_conditions.append({"question_id": {"$lte": filter_params.question_id_max}})
    
    if id_conditions:
        conditions.extend(id_conditions)
    
    # 조건 조합
    if len(conditions) > 1:
        if filter_params.operation == "$or":
            filter_dict = {"$or": conditions}
        else:
            filter_dict = {"$and": conditions}
    elif len(conditions) == 1:
        filter_dict = conditions[0]
    
    print(f"생성된 필터: {filter_dict}")
    
    # 검색 실행
    retriever = vector_store_with_summary.as_retriever(
        search_kwargs={"filter": filter_dict} if filter_dict else {}
    )
    
    results = retriever.invoke(query)
    return results

# RAG Chain

### 1) 참조 문서 없이 직접 답변을 생성

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt
template = '''Answer the question based only on the following context.

[Context]
{context}

[Question]
{question}

[Answer (in 한국어)]
'''

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)


# Chain 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
rag_chain.invoke(query)

'수원시에서 공급되는 주택의 해당 주택건설지역은 수원시의 행정구역에 해당됩니다.'

### 2) 참조 문서를 답변과 함께 반환

`(1) 문서와 포맷팅된 컨텍스트를 함께 반환하는 함수`


In [36]:
from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """
    # 문서 포맷팅 함수
    def format_docs(docs):
        return '\n\n'.join([d.page_content for d in docs])

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": question,  # 질문
        "context": format_docs(docs),   # 문서 포맷팅된 컨텍스트
        "source_documents": docs   # 검색된 문서 리스트
    }

`(2) 컨텍스트와 질문을 입력으로 받아 답변을 생성하는 함수`

In [37]:
from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """
    # 프롬프트 템플릿 설정
    template = """다음 컨텍스트를 기반으로 질문에 답변해주세요.

컨텍스트:
{context}

질문:
{question}

답변:"""

    prompt_template = ChatPromptTemplate.from_template(template)
    
    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = prompt_template | llm | StrOutputParser()
    
    # 답변 생성
    answer = answer_chain.invoke({
        "context": input_data["context"],
        "question": input_data["question"]
    })

    return {
        "answer": answer,  # 생성된 답변
        "source_documents": input_data["source_documents"]  # 소스 문서 정보
    }

`(3) RAG 체인 구성`

In [38]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    RunnableLambda(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': RunnableLambda(prompt_and_generate_answer), # 답변 생성
        'question': itemgetter("question"),
        "source_documents": itemgetter("source_documents")   # 소스 반환
    }
)

In [39]:
# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

답변: 수원시의 주택건설지역은 수원시의 행정구역에 해당합니다. 즉, 특별시·광역시·특별자치시·특별자치도 또는 시·군의 행정구역을 기준으로 할 때, 경기도 수원시에서 공급되는 주택의 해당 주택건설지역은 수원시 전역이 됩니다.

참조 문서:

문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


문서 2:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


문서 3:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.



### 3) 검색 문서 관련성 평가

`(1) 검색 문서와 질문 간의 관련성을 평가`


In [40]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")


문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes

문서 2:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes

문서 3:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: no


In [41]:
# gpt-4.1 모델 사용

llm_gpt4o = ChatOpenAI(
    model='gpt-4.1',
    temperature=0.1,
    top_p=0.9, 
)

chain = prompt | llm_gpt4o | StrOutputParser()    # gpt-4.1 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")


문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes

문서 2:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes

문서 3:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes


##### ***[실습] 문서 관련성 평가 체인을 구조화 출력으로 구현합니다.*** 

- pydantic schema 사용
- with_structured_output 함수 사용

In [42]:
from pydantic import BaseModel, Field

# 관련성 평가를 위한 출력 스키마 정의
class RelevanceOutput(BaseModel):
    is_relevant: bool = Field(description="문서가 질문과 관련있는지 여부")
    confidence: float = Field(description="관련성에 대한 확신도 (0.0-1.0)")
    reasoning: str = Field(description="관련성 판단에 대한 근거")

# 관련성 평가 프롬프트 템플릿
relevance_system_prompt = """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.
다음 평가 단계를 따르세요:

1. 질문의 핵심 요구사항을 파악하세요.
2. 컨텍스트의 내용을 분석하세요.
3. 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 판단하세요.
4. 관련성에 대한 확신도를 0.0부터 1.0 사이의 값으로 제시하세요.
5. 판단 근거를 명확히 설명하세요.

평가 기준:
- 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있으면 관련성이 높습니다.
- 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있으면 관련성이 중간 정도입니다.
- 컨텍스트가 질문과 완전히 무관한 내용을 담고 있으면 관련성이 없습니다.
"""

relevance_prompt = ChatPromptTemplate.from_messages([
    ("system", relevance_system_prompt),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

# 구조화된 출력을 위한 체인 생성
relevance_checker = (
    relevance_prompt | 
    llm.with_structured_output(RelevanceOutput)
)

# 문서 관련성 평가 함수
def check_document_relevance(doc, question):
    """문서의 관련성을 평가하는 함수"""
    return relevance_checker.invoke({
        "context": doc.page_content,
        "question": question
    })

# 사용 예시
test_query = "수원시의 주택건설지역은 어디에 해당하나요?"
test_docs = retriever.invoke(test_query)

for i, doc in enumerate(test_docs):
    relevance_result = check_document_relevance(doc, test_query)
    print(f"\n문서 {i+1} 관련성 평가:")
    print(f"관련 여부: {relevance_result.is_relevant}")
    print(f"확신도: {relevance_result.confidence}")
    print(f"평가 근거: {relevance_result.reasoning}")
    print("-" * 50)


문서 1 관련성 평가:
관련 여부: False
확신도: 0.9
평가 근거: 컨텍스트는 경기도 과천시 및 서울특별시, 인천광역시의 주택건설지역 범위에 대해 설명하고 있으나, 수원시의 주택건설지역에 대한 정보는 포함되어 있지 않습니다. 따라서 수원시의 주택건설지역에 대한 질문에 직접적으로 답변할 수 있는 정보가 없습니다.
--------------------------------------------------

문서 2 관련성 평가:
관련 여부: False
확신도: 0.9
평가 근거: 컨텍스트는 경기도 과천시와 서울특별시, 인천광역시의 주택건설지역 범위에 대해 설명하고 있으나, 수원시의 주택건설지역에 대한 정보는 포함하고 있지 않습니다. 따라서 수원시의 주택건설지역에 대한 질문에 직접적으로 답변할 수 있는 정보가 없습니다.
--------------------------------------------------

문서 3 관련성 평가:
관련 여부: False
확신도: 0.9
평가 근거: 컨텍스트는 경기도 과천시 및 서울특별시, 인천광역시의 주택건설지역 범위에 대해 설명하고 있으나, 수원시의 주택건설지역에 대한 정보는 포함하고 있지 않습니다. 따라서 수원시의 주택건설지역에 대해 답변하는 데 필요한 정보가 컨텍스트에 없습니다.
--------------------------------------------------


# Gradio 챗봇 인터페이스

In [43]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_sotre = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급지역
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역, 경기도 과천시, 특별시, 광역시, 특별자치시, 특별자치도, 행정구역, 주택 공급 지역
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

-

In [44]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class SearchResult:
    context: str
    source_documents: Optional[List]

class RAGSystem:
    def __init__(
            self, 
            llm: BaseChatModel, 
            eval_llm: BaseChatModel,
            retriever: VectorStoreRetriever
        ):
        if not llm:
            self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        else:
            self.llm = llm

        if not eval_llm:
            self.eval_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
        else:
            self.eval_llm = eval_llm

        if not retriever:
            raise ValueError("검색기(retriever)가 필요합니다.")
        else:
            self.retriever = retriever
        
    def _format_docs(self, docs: List) -> str:
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _format_source_documents(self, docs: Optional[List]) -> str:
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            if 'summary' in metadata:
                source_info.append(f"요약: {metadata['summary']}")
                
            formatted_docs.append(
                f"📚 참조 문서 {i}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def _check_relevance(self, docs: List, question: str) -> List:
        """문서의 관련성 확인"""

        relevant_docs = []

        if not docs:
            return relevant_docs
            
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

        다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

        1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
        2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?

        'Yes' 또는 'No'로만 답변하세요."""),
            ("human", """[컨텍스트]
        {context}

        [질문]
        {question}""")
        ])
        
        chain = prompt | self.eval_llm | StrOutputParser()

        for doc in docs:
            result = chain.invoke({
                "context": doc.page_content,
                "question": question
            }).lower()

            print(f"문서 {doc.metadata['question_id']} 관련성 확인 결과: {result}")
            print(f"문서 {doc.metadata['question_id']} 내용:")
            print(doc.page_content)
            print("-" * 50)

            if "yes" in result:
                relevant_docs.append(doc)
            
        return relevant_docs
    
    def search_documents(self, question: str) -> SearchResult:
        try:
            docs = retriever.invoke(question)
            print(f"검색된 문서 개수: {len(docs)}")
            relevant_docs = self._check_relevance(docs, question) 
            print(f"관련 문서 개수: {len(relevant_docs)}")
            
            return SearchResult(
                context=self._format_docs(relevant_docs) if relevant_docs else "관련 문서를 찾을 수 없습니다.",
                source_documents=relevant_docs,
            )
        except Exception as e:
            print(f"문서 검색 중 오류 발생: {e}")
            return SearchResult(
                context="문서 검색 중 오류가 발생했습니다.",
                source_documents=None,
            )
    
    def generate_answer(self, message: str, history: List) -> str:
        # 문서 검색
        search_result = self.search_documents(message)
        
        if not search_result.source_documents:
            return "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"
                    
        # 프롬프트 템플릿 설정
        prompt = ChatPromptTemplate.from_messages([
            ("system", """다음 지침을 따라 질문에 답변해주세요:
            1. 주어진 문서의 내용만을 기반으로 답변하세요.
            2. 문서에 명확한 근거가 없는 내용은 "근거 없음"이라고 답변하세요.
            3. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 답변하세요.
            4. 추측이나 일반적인 지식을 사용하지 마세요."""),
            ("human", "문서들:\n{context}\n\n질문: {question}")
        ])
        
        # RAG Chain 구성
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            # 답변 생성
            answer = chain.invoke({
                "context": search_result.context,
                "question": message
            })
            
            # 참조 문서 포맷팅 추가
            sources = self._format_source_documents(search_result.source_documents)
            return f"{answer}\n{sources}"
            
        except Exception as e:
            return f"답변 생성 중 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정

rag_system = RAGSystem(
    llm=ChatOpenAI(model="gpt-4.1", temperature=0),   
    eval_llm=ChatOpenAI(model="gpt-4.1", temperature=0),
    retriever=vector_store.as_retriever(search_kwargs={"k": 2})
)

demo = gr.ChatInterface(
    fn=rag_system.generate_answer,
    title="RAG QA 시스템",
    description="""
    질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
    모든 답변에는 참조한 문서의 출처가 표시됩니다.
    """,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="gray",
    ),
examples=[
    ["수원시의 주택건설지역은 어디에 해당하나요?"],
    ["무주택 세대에 대해서 설명해주세요."],
    ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
],
)

# 데모 실행
demo.launch()

/Users/kaydash/miniconda3/envs/modu/lib/python3.11/site-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [48]:
# Gradio 인터페이스 종료
demo.close()

---

# **[실습] 주택청약 FAQ 시스템 구현**

### **문제 설명**
이전 코드를 기반으로 주택청약 FAQ 시스템을 다음 요구사항에 맞춰 개선합니다. 

1. 응답 품질 향상 (1개 이상)
   - 생성된 답변의 품질을 평가 (답변이 불충분한 경우 예외 처리)
   - 관련성 높은 FAQ 문서 검색 (임베딩 모델, 청크 크기, 벡터 검색 방법 등)

2. 사용자 경험 개선 (1개 이상)
   - 대화 이력 관리 기능 추가 (요약, 트리밍 기능 등 고려)
   - 최근 대화 기반 컨텍스트 구성 
   - 사용자 프로필 기반 맞춤 응답

### **제약 조건**
- Gradio ChatInterface 사용
- RAG 구조 유지

In [49]:
# ==================== 고급 RAG 시스템 구현 ====================
# 하이브리드 검색, 질문 확장, 컨텍스트 최적화, 품질 평가 등 모든 개선사항 통합

import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from typing import List, Optional, Dict
from dataclasses import dataclass
from pydantic import BaseModel, Field
import sqlite3
import json
import time
import ast
import numpy as np
from datetime import datetime

# ==================== 1. 하이브리드 검색 구현 ====================

def create_hybrid_retriever(vector_store, documents):
    """BM25와 벡터 검색을 결합한 하이브리드 검색기"""
    # BM25 검색기 생성
    bm25_retriever = BM25Retriever.from_documents(documents)
    bm25_retriever.k = 5
    
    # 벡터 검색기
    vector_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    
    def hybrid_search(query, bm25_weight=0.3):
        # BM25 검색
        bm25_results = bm25_retriever.get_relevant_documents(query)
        # 벡터 검색
        vector_results = vector_retriever.get_relevant_documents(query)
        
        # 결과 결합 및 중복 제거
        seen_ids = set()
        hybrid_results = []
        
        # BM25 결과 추가
        for doc in bm25_results:
            doc_id = doc.metadata.get('question_id', hash(doc.page_content))
            if doc_id not in seen_ids:
                seen_ids.add(doc_id)
                doc.metadata['score'] = 1.0  # BM25 점수 추가
                hybrid_results.append(doc)
        
        # 벡터 검색 결과 추가 (중복 제외)
        for doc in vector_results:
            doc_id = doc.metadata.get('question_id', hash(doc.page_content))
            if doc_id not in seen_ids:
                seen_ids.add(doc_id)
                doc.metadata['score'] = 0.8  # 벡터 검색 점수
                hybrid_results.append(doc)
        
        # 점수별 정렬
        hybrid_results.sort(key=lambda x: x.metadata.get('score', 0), reverse=True)
        return hybrid_results[:5]
    
    return hybrid_search

# ==================== 2. 질문 분류 및 확장 시스템 ====================

class QuestionClassifier(BaseModel):
    category: str = Field(description="질문 카테고리: 청약, 주택, 대출, 정책, 기타")
    keywords: List[str] = Field(description="추출된 핵심 키워드")
    complexity: str = Field(description="질문 복잡도: 간단, 보통, 복잡")
    intent: str = Field(description="질문 의도: 정보요청, 절차문의, 자격확인, 계산요청 등")

class QueryProcessor:
    def __init__(self, llm):
        self.llm = llm
        self.classification_chain = self._create_classification_chain()
        self.expansion_chain = self._create_expansion_chain()
    
    def _create_classification_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주택청약 관련 질문을 분석하여 다음 정보를 추출하세요:
            1. 카테고리: 청약, 주택, 대출, 정책, 기타 중 하나
            2. 핵심 키워드: 중요한 용어들을 추출
            3. 복잡도: 간단(일반 정보), 보통(절차 설명), 복잡(복합 조건) 중 하나
            4. 의도: 정보요청, 절차문의, 자격확인, 계산요청 중 하나"""),
            ("human", "질문: {query}")
        ])
        
        return prompt | self.llm.with_structured_output(QuestionClassifier)
    
    def _create_expansion_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 질문과 관련된 유사한 질문들을 3개 생성하세요.
            생성된 질문들은 원본 질문과 관련이 있으면서도 약간 다른 각도에서 접근해야 합니다.
            각 질문은 한 줄로 작성하고, 번호를 붙이지 마세요."""),
            ("human", "원본 질문: {query}\n분류 정보: {classification}")
        ])
        
        return prompt | self.llm | StrOutputParser()
    
    def process_query(self, query: str) -> Dict:
        # 1. 질문 분류
        classification = self.classification_chain.invoke({"query": query})
        
        # 2. 질문 확장
        expansion_result = self.expansion_chain.invoke({
            "query": query,
            "classification": classification
        })
        
        # 확장된 질문들 파싱
        expanded_queries = [q.strip() for q in expansion_result.split('\n') if q.strip()]
        
        return {
            "category": classification.category,
            "keywords": classification.keywords,
            "complexity": classification.complexity,
            "intent": classification.intent,
            "expanded_queries": expanded_queries
        }

# ==================== 3. 컨텍스트 최적화 시스템 ====================

class ContextOptimizer:
    def __init__(self, llm):
        self.llm = llm
        self.compression_chain = self._create_compression_chain()
    
    def _create_compression_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 문서들에서 질문과 관련된 핵심 정보만 추출하여 압축하세요.
            - 질문 답변에 필요한 핵심 정보만 포함
            - 중복되는 내용은 제거
            - 구체적인 수치, 조건, 절차는 정확히 유지
            - 압축된 내용은 자연스럽게 읽힐 수 있도록 구성"""),
            ("human", """질문: {query}
            
            문서 내용:
            {documents}
            
            압축된 컨텍스트:""")
        ])
        
        return prompt | self.llm | StrOutputParser()
    
    def compress_context(self, documents: List, query: str, max_length: int = 2000) -> str:
        if not documents:
            return ""
        
        # 문서 내용 결합
        doc_text = "\n\n".join([f"문서 {i+1}: {doc.page_content}" for i, doc in enumerate(documents)])
        
        # 길이가 짧으면 압축하지 않음
        if len(doc_text) <= max_length:
            return doc_text
        
        # 컨텍스트 압축
        compressed = self.compression_chain.invoke({
            "query": query,
            "documents": doc_text
        })
        
        return compressed

# ==================== 4. 답변 품질 평가 시스템 ====================

class AnswerQualityAdvanced(BaseModel):
    completeness_score: float = Field(description="답변 완성도 점수 (0.0-1.0)")
    clarity_score: float = Field(description="답변 명확성 점수 (0.0-1.0)")
    accuracy_score: float = Field(description="답변 정확성 점수 (0.0-1.0)")
    is_sufficient: bool = Field(description="답변이 충분한지 여부")
    improvement_suggestions: List[str] = Field(description="개선 제안사항")
    reasoning: str = Field(description="평가 근거")

class AnswerEvaluator:
    def __init__(self, llm):
        self.llm = llm
        self.evaluation_chain = self._create_evaluation_chain()
    
    def _create_evaluation_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주택청약 관련 답변의 품질을 평가하세요.
            
            평가 기준:
            1. 완성도 (0.0-1.0): 질문에 대한 답변이 얼마나 완전한가?
            2. 명확성 (0.0-1.0): 답변이 얼마나 이해하기 쉬운가?
            3. 정확성 (0.0-1.0): 제공된 정보가 얼마나 정확한가?
            4. 충분성: 답변이 질문자의 요구를 충족하는가?
            5. 개선사항: 구체적인 개선 제안
            6. 근거: 평가 이유
            
            점수가 0.7 미만이면 답변 개선이 필요합니다."""),
            ("human", """질문: {query}
            
            답변: {answer}
            
            참조 컨텍스트: {context}
            
            답변 품질을 평가하세요:""")
        ])
        
        return prompt | self.llm.with_structured_output(AnswerQualityAdvanced)
    
    def evaluate(self, query: str, answer: str, context: str) -> AnswerQualityAdvanced:
        return self.evaluation_chain.invoke({
            "query": query,
            "answer": answer,
            "context": context
        })

# ==================== 5. 대화 이력 관리 시스템 ====================

class ConversationDB:
    def __init__(self, db_path="chat_history.db"):
        self.db_path = db_path
        self.init_db()
    
    def init_db(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id TEXT,
                query TEXT,
                answer TEXT,
                context TEXT,
                relevance_score REAL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def save_conversation(self, user_id: str, query: str, answer: str, context: str, relevance_score: float):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO conversations (user_id, query, answer, context, relevance_score)
            VALUES (?, ?, ?, ?, ?)
        ''', (user_id, query, answer, context, relevance_score))
        
        conn.commit()
        conn.close()
    
    def get_user_history(self, user_id: str, limit: int = 5):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT query, answer, timestamp FROM conversations
            WHERE user_id = ?
            ORDER BY timestamp DESC
            LIMIT ?
        ''', (user_id, limit))
        
        results = cursor.fetchall()
        conn.close()
        
        return results

# ==================== 6. 통합 RAG 시스템 ====================

class AdvancedRAGSystem:
    def __init__(self, vector_store, llm, eval_llm):
        self.vector_store = vector_store
        self.llm = llm
        self.eval_llm = eval_llm
        
        # 컴포넌트 초기화
        self.query_processor = QueryProcessor(llm)
        self.context_optimizer = ContextOptimizer(llm)
        self.answer_evaluator = AnswerEvaluator(eval_llm)
        self.chat_db = ConversationDB()
        
        # 하이브리드 검색기 초기화
        docs = list(vector_store.get()['documents'])
        doc_objects = [Document(page_content=doc) for doc in docs]
        self.advanced_retriever = create_hybrid_retriever(vector_store, doc_objects)
        
        # 답변 생성 체인 초기화
        self.answer_chain = self._create_answer_chain()
        self.refinement_chain = self._create_refinement_chain()
    
    def _create_answer_chain(self):
        template = """당신은 주택청약 전문가입니다. 주어진 정보를 바탕으로 정확하고 도움이 되는 답변을 제공하세요.

참조 정보:
{context}

질문: {query}

답변 작성 가이드라인:
1. 정확하고 구체적인 정보 제공
2. 절차나 조건이 있는 경우 단계적으로 설명
3. 관련 법규나 정책이 있으면 언급
4. 추가로 도움이 될 만한 정보도 포함
5. 이해하기 쉽게 구조화된 답변

답변:"""
        
        prompt = ChatPromptTemplate.from_template(template)
        return prompt | self.llm | StrOutputParser()
    
    def _create_refinement_chain(self):
        template = """다음 답변을 개선해주세요.

원본 질문: {query}
초기 답변: {initial_answer}
참조 컨텍스트: {context}
개선 필요 사항: {critique}

개선된 답변을 작성하세요:
1. 부족한 정보를 보완
2. 명확성을 높임
3. 구체적인 예시나 수치 추가
4. 더 구조화된 형태로 재작성

개선된 답변:"""
        
        prompt = ChatPromptTemplate.from_template(template)
        return prompt | self.llm | StrOutputParser()
    
    def _evaluate_answer_quality(self, query: str, answer: str, context: str) -> AnswerQualityAdvanced:
        """답변 품질 평가"""
        return self.answer_evaluator.evaluate(query, answer, context)
    
    def _refine_answer(self, query: str, initial_answer: str, context: str, critique: str) -> str:
        """답변 개선"""
        return self.refinement_chain.invoke({
            "query": query,
            "initial_answer": initial_answer,
            "context": context,
            "critique": critique
        })
    
    def _format_source_documents(self, docs: Optional[List], relevance_scores: Optional[List] = None) -> str:
        """참조 문서 포맷팅 (관련성 점수 포함)"""
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            
            # 관련성 점수 추가
            relevance_info = ""
            if relevance_scores and i-1 < len(relevance_scores):
                relevance_info = f" (관련도: {relevance_scores[i-1]:.2f})"
                
            formatted_docs.append(
                f"📚 참조 문서 {i}{relevance_info}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def generate_answer(self, query, user_id: str = "default"):
        """향상된 답변 생성 파이프라인"""
        # 1. 질문 처리 및 확장
        processed_query = self.query_processor.process_query(query)
        
        # 2. 최적의 검색 질문 선택
        search_queries = [query] + processed_query["expanded_queries"][:2]
        
        # 3. 다중 검색 실행 및 결과 결합
        all_docs = []
        for q in search_queries:
            docs = self.advanced_retriever.retrieve(q)
            all_docs.extend(docs)
        
        # 중복 제거 (question_id 기준)
        unique_docs = []
        seen_ids = set()
        for doc in all_docs:
            doc_id = doc.metadata.get('question_id')
            if doc_id not in seen_ids:
                seen_ids.add(doc_id)
                unique_docs.append(doc)
        
        # 4. 컨텍스트 최적화
        optimized_context = self.context_optimizer.compress_context(unique_docs, query)
        
        # 5. 초기 답변 생성
        initial_answer = self.answer_chain.invoke({
            "query": query,
            "context": optimized_context
        })
        
        # 6. 답변 품질 평가
        quality_result = self._evaluate_answer_quality(query, initial_answer, optimized_context)
        
        # 7. 품질이 낮은 경우 답변 개선
        if not quality_result.is_sufficient or quality_result.completeness_score < 0.7:
            refined_answer = self._refine_answer(
                query, initial_answer, optimized_context, quality_result.reasoning
            )
        else:
            refined_answer = initial_answer
        
        # 8. 대화 이력 저장
        avg_relevance = 0.8  # 기본값
        if unique_docs:
            self.chat_db.save_conversation(user_id, query, refined_answer, optimized_context, avg_relevance)
        
        # 9. 소스 문서 정보 추가
        source_info = self._format_source_documents(unique_docs)
        
        # 10. 품질 정보 추가
        quality_info = f"\n\n📊 답변 품질: 완성도 {quality_result.completeness_score:.1f}/1.0, 명확성 {quality_result.clarity_score:.1f}/1.0"
        
        return {
            "answer": refined_answer + source_info + quality_info,
            "sources": source_info,
            "metadata": {
                "query_category": processed_query["category"],
                "keywords": processed_query["keywords"],
                "context_length": len(optimized_context),
                "quality": quality_result
            }
        }

# ==================== 8. 시스템 초기화 및 인터페이스 ====================

# 모델 및 벡터 저장소 초기화
embeddings_advanced = OpenAIEmbeddings(model="text-embedding-3-small")
llm_advanced = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
eval_llm_advanced = ChatOpenAI(model="gpt-4o", temperature=0)

vector_store_advanced = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings_advanced,
)

# 고급 RAG 시스템 초기화
advanced_rag_system = AdvancedRAGSystem(vector_store_advanced, llm_advanced, eval_llm_advanced)

# 세션 상태 관리
class SessionState:
    def __init__(self):
        self.conversation_history = []
        self.last_query = None
        self.last_answer = None

session = SessionState()

# 메인 답변 생성 함수 (수정됨)
def generate_response(message, history):
    if not message.strip():
        return history + [("질문을 입력해주세요.", None)]
    
    session.last_query = message
    session.conversation_history.append({"role": "user", "content": message})
    
    try:
        # 고급 답변 생성
        result = advanced_rag_system.generate_answer(message, "default_user")
        answer = result["answer"]
        
        session.last_answer = answer
        session.conversation_history.append({"role": "assistant", "content": answer})
        
        # 중요: history를 업데이트한 형태로 반환
        return history + [(message, answer)]
    except Exception as e:
        error_msg = f"답변 생성 중 오류가 발생했습니다: {str(e)}"
        return history + [(message, error_msg)]

# 후속 질문 제안 함수
def suggest_followup_questions():
    if not session.last_query or not session.last_answer:
        return ["청약통장의 종류는 무엇인가요?", "주택청약 자격 조건은 어떻게 되나요?", "청약 가점제란 무엇인가요?"]
    
    followup_prompt = ChatPromptTemplate.from_messages([
        ("system", """이전 대화 내용을 바탕으로 사용자가 다음에 물어볼 만한 관련 질문 3개를 생성하세요.
        질문은 간결하고 구체적이어야 하며, 이전 대화에서 미처 다루지 못한 관련 주제를 다루어야 합니다."""),
        ("human", """이전 질문: {last_query}
        
        이전 답변: {last_answer}
        
        후속 질문 3개를 생성하세요:""")
    ])
    
    followup_chain = followup_prompt | llm_advanced | StrOutputParser()
    
    try:
        result = followup_chain.invoke({
            "last_query": session.last_query,
            "last_answer": session.last_answer
        })
        
        # 결과 파싱
        questions = [q.strip() for q in result.split('\n') if q.strip()]
        questions = [q.split('. ', 1)[-1].split('- ', 1)[-1] for q in questions]
        
        return questions[:3]
    except:
        return ["청약통장의 종류는 무엇인가요?", "주택청약 자격 조건은 어떻게 되나요?", "청약 가점제란 무엇인가요?"]

# Gradio 인터페이스 구성
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as demo_advanced:
    gr.Markdown("""
    # 🏠 고급 주택청약 FAQ 시스템
    
    **주요 개선사항:**
    - 하이브리드 검색 (키워드 + 의미 기반)
    - 질문 확장 및 분류
    - 컨텍스트 최적화
    - 답변 품질 자동 평가 및 개선
    - 대화 이력 기반 맞춤형 응답
    - 참조 문서 관련성 점수 표시
    """)
    
    with gr.Row():
        with gr.Column(scale=7):
            chatbot = gr.Chatbot(height=450, type='messages')
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="질문을 입력하세요...",
                    label="질문",
                    show_label=False
                )
                submit_btn = gr.Button("질문하기")
            
            with gr.Row():
                refresh_btn = gr.Button("추천 질문 보기")
                suggestions_output = gr.Textbox(
                    label="추천 질문",
                    lines=3,
                    visible=False
                )
        
        with gr.Column(scale=3):
            with gr.Accordion("시스템 정보", open=False):
                gr.Markdown("""
                ### 시스템 기능
                
                - 🔍 하이브리드 검색 (BM25 + 벡터)
                - 📝 질문 확장 및 분류
                - 🎯 컨텍스트 최적화 및 압축
                - ⭐ 답변 품질 자동 평가
                - 💬 대화 이력 기반 개인화
                - 📊 성능 모니터링
                
                ### 사용 팁
                
                - 구체적인 질문이 더 정확한 답변을 얻을 수 있습니다
                - 관련 후속 질문을 이어서 물어보면 더 자세한 정보를 얻을 수 있습니다
                - "추천 질문 보기"를 클릭하면 관련 질문이 표시됩니다
                """)
    
    # 이벤트 핸들러
    msg.submit(generate_response, [msg, chatbot], [chatbot]).then(
        lambda: "", None, [msg]
    )
    
    submit_btn.click(generate_response, [msg, chatbot], [chatbot]).then(
        lambda: "", None, [msg]
    )
    
    def update_suggestions():
        questions = suggest_followup_questions()
        formatted_questions = "\n".join([f"• {q}" for q in questions])
        return gr.Textbox.update(
            value=formatted_questions,
            visible=True
        )
    
    refresh_btn.click(update_suggestions, None, [suggestions_output])

print("🚀 고급 주택청약 FAQ 시스템이 성공적으로 초기화되었습니다!")
print("주요 개선사항:")
print("- 하이브리드 검색 (BM25 + 벡터 검색)")
print("- 질문 확장 및 분류")
print("- 컨텍스트 최적화")
print("- 답변 품질 자동 평가 및 개선")
print("- 대화 이력 기반 맞춤형 응답")
print("- 참조 문서 관련성 점수 표시")

# 시스템 실행
demo_advanced.launch(share=True)

🚀 고급 주택청약 FAQ 시스템이 성공적으로 초기화되었습니다!
주요 개선사항:
- 하이브리드 검색 (BM25 + 벡터 검색)
- 질문 확장 및 분류
- 컨텍스트 최적화
- 답변 품질 자동 평가 및 개선
- 대화 이력 기반 맞춤형 응답
- 참조 문서 관련성 점수 표시
* Running on local URL:  http://127.0.0.1:7862
* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://45f5ed714b9dbdf545.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://45f5ed714b9dbdf545.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
